# **Building Image Captioning Model using CNN Encoder and RNN Decoder**

# Importing all the libraries

In [1]:
#installing pycocoevalcap from github for CIDEr and ROUGE-L metrics
!pip install git+https://github.com/salaniz/pycocoevalcap

  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-zndykp1w
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-zndykp1w
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 2.8 MB/s eta 0:00:00
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=02c1467a2508769f3037123c8a7141a7e84f2621b6adf2fef588b4221b88f113
  Stored in directory: /tmp/pip-ephem-wheel-cache-t0sex1om/wheels/43/54/73/3e2c6d4ace7657958cde52ac6fd47b342cd4aae5a7aa4fcbf9
Successfully built pycocoevalcap


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models
import os
import pandas as pd
import spacy
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import torch.optim as optim
import statistics
from tqdm import tqdm
import pycocoevalcap
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from torchmetrics.text import BLEUScore

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing and Data Loading

In [4]:
spacy_eng = spacy.load("en_core_web_sm")

class vocabularry:
    def __init__(self, fr_th):
        self.int_to_str = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.str_to_int = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.fr_th = fr_th

    def __len__(self):
        return len(self.int_to_str)

    @staticmethod
    def tok_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabularry(self, sen_lst):
        frequ = {}
        idx = 4

        for sent in sen_lst:
            for word in self.tok_eng(sent):
                if word not in frequ:
                    frequ[word] = 1

                else:
                    frequ[word] += 1

                if frequ[word] == self.fr_th:
                    self.str_to_int[word] = idx
                    self.int_to_str[idx] = word
                    idx += 1

    def numriclise(self, text):
        tokenized_text = self.tok_eng(text)

        return [
            self.str_to_int[token] if token in self.str_to_int else self.str_to_int["<UNK>"]
            for token in tokenized_text
        ]


class ImgCapDataset(Dataset):
    def __init__(self, root_dir, cap_file, trans=None, fr_th=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(cap_file)
        self.trans = trans
        self.imgs = self.df["filename"]
        self.caps = self.df["caption"]
        self.vocab = vocabularry(fr_th)
        self.vocab.build_vocabularry(self.caps.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        cap = self.caps[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        if self.trans is not None:
            img = self.trans(img)

        numriclised_cap = [self.vocab.str_to_int["<SOS>"]]
        numriclised_cap += self.vocab.numriclise(cap)
        numriclised_cap.append(self.vocab.str_to_int["<EOS>"])

        return img, torch.tensor(numriclised_cap), cap, img_id


class mycoll:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)
        caps = [item[2] for item in batch]
        img_ids = [item[3] for item in batch]
        return imgs, targets, caps, img_ids


def get_loamder(
    root_folder,
    annot_file,
    trans,
    batch_size=32,
    num_workers=4,
    shuffle=True,
    pin_memory=True,
):
    dataset = ImgCapDataset(root_folder, annot_file, trans=trans)

    pad_idx = dataset.vocab.str_to_int["<PAD>"]

    loamder = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=mycoll(pad_idx=pad_idx),
    )

    return loamder, dataset

In [5]:
# Transforms used for training data
train_transform = transforms.Compose(
    [
        transforms.Resize((300, 300)),
        transforms.RandomCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    ]
)
# Transforms used for testing data
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    ]
)
# Loading Datasets
train_loamder, dataset = get_loamder(
    root_folder="/kaggle/input/image-captioning/custom_captions_dataset/train",
    annot_file="/kaggle/input/image-captioning/custom_captions_dataset/train.csv",
    trans=train_transform,
    num_workers=2,
)

val_loamder, val_dataset = get_loamder(root_folder="/kaggle/input/image-captioning/custom_captions_dataset/val",
    annot_file="/kaggle/input/image-captioning/custom_captions_dataset/val.csv",
    trans=transform,batch_size=1,shuffle=False,
    num_workers=2,)

test_loamder, test_dataset = get_loamder(root_folder="/kaggle/input/image-captioning/custom_captions_dataset/test",
    annot_file="/kaggle/input/image-captioning/custom_captions_dataset/test.csv",
    trans=transform,batch_size=1,shuffle=False,
    num_workers=2,)

# Building the Image Captioning Model Class

In [6]:
# Class Implementation of CNN based encoder for the Image Captioning model
class image_encoder(nn.Module):
    def __init__(self, emb_size):
        super(image_encoder, self).__init__()
        self.res_nett_50 = models.resnet50(weights='DEFAULT')
        self.res_nett_50.fc = nn.Linear(self.res_nett_50.fc.in_features, emb_size)
        self.RELUU = nn.ReLU()
        self.dr = nn.Dropout(0.5)

    def forward(self, imgs):
        feat_encoderr = self.res_nett_50(imgs)
        return self.dr(self.RELUU(feat_encoderr))

# Class Implementation of RNN based decoder for the Image Captioning model
class cap_decoder(nn.Module):
    def __init__(self, emb_size, hid_size, voc_size, num_layers):
        super(cap_decoder, self).__init__()
        self.emb = nn.Embedding(voc_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hid_size, num_layers)
        self.linear = nn.Linear(hid_size, voc_size)
        self.dr = nn.Dropout(0.5)

    def forward(self, feat_encoderr, captions):
        embeddings = self.dr(self.emb(captions))
        embeddings = torch.cat((feat_encoderr.unsqueeze(0), embeddings), dim=0)
        hid, _ = self.lstm(embeddings)
        out = self.linear(hid)
        return out

# Class Implementation of Image Captioning model using image_encoder, cap_decoder classes
class Image_Captioner(nn.Module):
    def __init__(self, emb_size, hid_size, voc_size, num_layers):
        super(Image_Captioner, self).__init__()
        self.cap_encoder = image_encoder(emb_size)
        self.cap_decoder = cap_decoder(emb_size, hid_size, voc_size, num_layers)

    def forward(self, imgs, captions):
        feat_encoderr = self.cap_encoder(imgs)
        out = self.cap_decoder(feat_encoderr, captions)
        return out

    def cap_img(self, image, vocab, m_len=50):
        res_cap = []

        with torch.no_grad():
            x = self.cap_encoder(image).unsqueeze(0)
            sts = None

            for _ in range(m_len):
                hid, sts = self.cap_decoder.lstm(x, sts)
                out = self.cap_decoder.linear(hid.squeeze(0))
                predi = out.argmax(1)
                res_cap.append(predi.item())
                x = self.cap_decoder.emb(predi).unsqueeze(0)

                if vocab.int_to_str[predi.item()] == "<EOS>":
                    break

        return [vocab.int_to_str[idx] for idx in res_cap]

In [7]:
# Function to save model checkpoints
def sve_ckpt(st, filename):
    torch.save(st, filename)

# Training the Image Captioning model

In [8]:
warnings.filterwarnings("ignore")
# comp_cider function is used to calculate CIDEr metric score
def comp_cider(ref, hyp):
    scorer = Cider()
    score, _ = scorer.compute_score(ref, hyp)
    return score
# comp_bleu function is defined to calculate BLEU metric score
def comp_bleu(ref, hyp):
    scorer = BLEUScore(n_gram=1)
    score = scorer(ref, hyp)
    return score.numpy()
# comp_rougel function is utilized to calculate ROUGE-L metric score
def comp_rougel(ref, hyp):
    scorer = Rouge()
    score, _ = scorer.compute_score(ref, hyp)
    return score

torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_model = False

emb_size = 512 # Embedding size
hid_size = 512 # Hidden size
voc_size = len(dataset.vocab) # size of the vocabulary
num_layers = 2 # number of LSTM layers in the RNN based Encoder
l_r = 3e-4 # Learning Rate
n_epoc = 300 # number of epochs

step = 0
# Creating an instance of the model
model = Image_Captioner(emb_size, hid_size, voc_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.str_to_int["<PAD>"])
optimiz = optim.Adam(model.parameters(), lr=l_r)
# For fine tuning the RESNET-50 model used in the CNN-Encoder
for name, params in model.cap_encoder.res_nett_50.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        params.requires_grad = True
    else:
        params.requires_grad = False

# Training the model for 300 epochs
for epoch in range(1,n_epoc+1):
    model.train()

    for idx, (imgs, captions, str_cap, img_ids) in tqdm(
        enumerate(train_loamder), total=len(train_loamder), leave=False
    ):
        imgs = imgs.to(device)
        captions = captions.to(device)

        out = model(imgs, captions[:-1])
        loss = criterion(
            out.reshape(-1, out.shape[2]), captions.reshape(-1)
        )

        step += 1

        optimiz.zero_grad()
        loss.backward(loss)
        optimiz.step()
    # Saving the model after completing training for 300 epochs
    if epoch == 300:
        save_model = True
    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimiz.state_dict(),
            "step": step,
        }
        sve_ckpt(checkpoint,filename=f"{epoch}_my_ckpt.pth.tar")
    model.eval()
    ref = {}
    hyp = {}
    bleu_ = []
    for idx, (img, captions, str_cap, img_id) in enumerate(val_loamder):
        img = img
        ref[idx] = str_cap
        a = model.cap_img(img.to(device), dataset.vocab)
        s = ""
        s += a[1]
        for i in range(2,len(a)):
            if a[i] == '.':
                s += a[i]
            elif a[i] == "<EOS>":
                continue
            else:
                s += " "
                s += a[i]
        hyp[idx] = [s]
        bleu_.append(comp_bleu([s],[str_cap]))
    print(f"CIDEr score of validation set for epoch {epoch}: ", comp_cider(ref,hyp))
    print(f"BLEU score of validation set for epoch {epoch}: ", sum(bleu_)/len(bleu_))
    print(f"ROUGE-l score of validation set for epoch {epoch}: ", comp_rougel(ref,hyp))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 163MB/s]


CIDEr score of validation set for epoch 1:  0.00018743027402933385
BLEU score of validation set for epoch 1:  0.04845521201027152
ROUGE-l score of validation set for epoch 1:  0.06013898561582912


CIDEr score of validation set for epoch 2:  0.004306340142536144
BLEU score of validation set for epoch 2:  0.19007045763542768
ROUGE-l score of validation set for epoch 2:  0.215630505038662


CIDEr score of validation set for epoch 3:  0.007745061190899408
BLEU score of validation set for epoch 3:  0.17459248341954695
ROUGE-l score of validation set for epoch 3:  0.15933326724701352


CIDEr score of validation set for epoch 4:  0.010752358922430169
BLEU score of validation set for epoch 4:  0.19333760553762222
ROUGE-l score of validation set for epoch 4:  0.2029049441080632


...

CIDEr score of validation set for epoch 296:  0.05291506877591479
BLEU score of validation set for epoch 296:  0.22444914583358952
ROUGE-l score of validation set for epoch 296:  0.2079058100332403


CIDEr score of validation set for epoch 297:  0.0525633227021643
BLEU score of validation set for epoch 297:  0.22081048264075662
ROUGE-l score of validation set for epoch 297:  0.20524460034771053


CIDEr score of validation set for epoch 298:  0.05690311634189152
BLEU score of validation set for epoch 298:  0.22120092059490537
ROUGE-l score of validation set for epoch 298:  0.2053246275656473


CIDEr score of validation set for epoch 299:  0.056357483459394
BLEU score of validation set for epoch 299:  0.2238893462345004
ROUGE-l score of validation set for epoch 299:  0.20658581660703665


CIDEr score of validation set for epoch 300:  0.05621605964931168
BLEU score of validation set for epoch 300:  0.21955924175274075
ROUGE-l score of validation set for epoch 300:  0.20633162304753447


# Testing the Image Captioning Model

In [9]:
warnings.filterwarnings("ignore")
model.eval()
ref = {}
hyp = {}
bleu_ = []
for idx, (img, captions, str_cap, img_id) in enumerate(test_loamder):
    img = img
    ref[idx] = str_cap
    a = model.cap_img(img.to(device), dataset.vocab)
    s = ""
    s += a[1]
    for i in range(2,len(a)):
        if a[i] == '.':
            s += a[i]
        elif a[i] == "<EOS>":
            continue
        else:
            s += " "
            s += a[i]
    hyp[idx] = [s]
    bleu_.append(comp_bleu([s],[str_cap]))
    print("Correct Caption:", ref[idx][0])
    print("Predicted Caption:", hyp[idx][0])
print("CIDEr score of test set: ", comp_cider(ref,hyp))
print("BLEU score of test set: ", sum(bleu_)/len(bleu_))
print("ROUGE-l score of test set: ", comp_rougel(ref,hyp))

Correct Caption: A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. 
Predicted Caption: a car is passing through an intersection at <UNK> street. two overhead street lights are showing green.   a white sign is showing on the street. two white lines are hanging on the street. two trucks are parked on the street.
Correct Caption: A person is skiing through the snow. There is loose snow all around them from him jumping. The person is wearing a yellow snow suit. The person is holding two ski poles in their hands. 
Predicted Caption: a man is skiing down a hill. his ski rod is in the air and there are a bunch of mountains in the distance. the sky is very clear and the clouds are extremely low.
Correct Caption: There is a bed in a room against a wall. There is a brown blanket on top of the bed. There is a small brown book shelf next to the bed. There is a picture hanging 